# Imports and Paths

In [1]:
import urllib
from bs4 import BeautifulSoup, Comment
import os
import pandas as pd
from plotnine import *
import plotnine
import time
import json

plotnine.options.figure_size = (12, 8)
from pandas.api.types import is_string_dtype
pd.options.display.max_columns=150
pd.options.display.max_rows=100

In [2]:
PATH = '/data/msnow/football/'

# Determine the urls for each games

In [3]:
bs_pg = 'https://www.pro-football-reference.com/years/'

In [6]:
yr = 2017
week = 1
pg_url = f'{bs_pg}{yr}/week_{week}.htm'

In [7]:
pg = urllib.request.urlopen(pg_url)
soup = BeautifulSoup(pg, 'html.parser')

In [13]:
links[0]

'/boxscores/201709070nwe.htm'

In [14]:
summ = soup.find('div',{'class':'game_summaries'})
links = summ.find_all('td',{'class':'gamelink'})
game_urls = []
game_urls_bs = 'https://www.pro-football-reference.com/'
for game in links:
    game_str = game.a['href']
    game_urls.append(f'{game_urls_bs}{game_str}')
game_urls

['https://www.pro-football-reference.com//boxscores/201709070nwe.htm',
 'https://www.pro-football-reference.com//boxscores/201709100det.htm',
 'https://www.pro-football-reference.com//boxscores/201709100cle.htm',
 'https://www.pro-football-reference.com//boxscores/201709100cin.htm',
 'https://www.pro-football-reference.com//boxscores/201709100chi.htm',
 'https://www.pro-football-reference.com//boxscores/201709100was.htm',
 'https://www.pro-football-reference.com//boxscores/201709100buf.htm',
 'https://www.pro-football-reference.com//boxscores/201709100oti.htm',
 'https://www.pro-football-reference.com//boxscores/201709100htx.htm',
 'https://www.pro-football-reference.com//boxscores/201709100ram.htm',
 'https://www.pro-football-reference.com//boxscores/201709100gnb.htm',
 'https://www.pro-football-reference.com//boxscores/201709100sfo.htm',
 'https://www.pro-football-reference.com//boxscores/201709100dal.htm',
 'https://www.pro-football-reference.com//boxscores/201709110min.htm',
 'http

In [15]:
def game_urls(yr,wk):
    bs_pg = 'https://www.pro-football-reference.com/years/'
    pg_url = f'{bs_pg}{yr}/week_{wk}.htm'
    pg = urllib.request.urlopen(pg_url)
    soup = BeautifulSoup(pg, 'html.parser')
    summ = soup.find('div',{'class':'game_summaries'})
    links = summ.find_all('td',{'class':'gamelink'})
    game_urls = []
    game_urls_bs = 'https://www.pro-football-reference.com/'
    for game in links:
        game_str = game.a['href']
        game_urls.append(f'{game_urls_bs}{game_str}')
    return game_urls

In [16]:
gurl = game_urls(2017,1)
gurl

['https://www.pro-football-reference.com//boxscores/201709070nwe.htm',
 'https://www.pro-football-reference.com//boxscores/201709100det.htm',
 'https://www.pro-football-reference.com//boxscores/201709100cle.htm',
 'https://www.pro-football-reference.com//boxscores/201709100cin.htm',
 'https://www.pro-football-reference.com//boxscores/201709100chi.htm',
 'https://www.pro-football-reference.com//boxscores/201709100was.htm',
 'https://www.pro-football-reference.com//boxscores/201709100buf.htm',
 'https://www.pro-football-reference.com//boxscores/201709100oti.htm',
 'https://www.pro-football-reference.com//boxscores/201709100htx.htm',
 'https://www.pro-football-reference.com//boxscores/201709100ram.htm',
 'https://www.pro-football-reference.com//boxscores/201709100gnb.htm',
 'https://www.pro-football-reference.com//boxscores/201709100sfo.htm',
 'https://www.pro-football-reference.com//boxscores/201709100dal.htm',
 'https://www.pro-football-reference.com//boxscores/201709110min.htm',
 'http

# Extract the play by play for each game

In [17]:
pg_url = 'https://www.pro-football-reference.com/boxscores/201709070nwe.htm'
pg = urllib.request.urlopen(pg_url)
soup = BeautifulSoup(pg, 'html.parser')

In [18]:
soup.find('table', {'id':'pbp'})

In [19]:
comments=soup.find_all(string=lambda text:isinstance(text,Comment))

In [37]:
for v in comments:
    if v.find("div_pbp") > -1:
        break

In [136]:
header = soup.find('h1').string.split(' -')[0]
header.split(' at ')
# header.split('-')[0]

['Kansas City Chiefs', 'New England Patriots ']

In [162]:
def pbp_extract(pg_url, yr, wk):
    pg = urllib.request.urlopen(pg_url)
    soup = BeautifulSoup(pg, 'html.parser')
    header = soup.find('h1').string.split(' -')[0]
    header = header.split(' at ')
    comments=soup.find_all(string=lambda text:isinstance(text,Comment))
    for v in comments:
        if v.find("div_pbp") > -1:
            break
    table = BeautifulSoup(str(v), 'lxml')
    rows = table.find_all('tr')
    quarter = 1
    pbp = []
    for row in rows[1:]:
        tmp_dict = {}
        cells = row.find_all('td')
        if len(cells) == 1:
            try:
                quarter = int(cells[0].string[0])
            except:
                quarter = cells[0].string
        else:
            for cell in cells:
                if cell['data-stat'] == 'detail':
                    val =  ''.join([x.string for x in cell.contents if x.string is not None])
                    tmp_dict[cell['data-stat']] = val
                else:
                    val = cell.string
                    try:
                        tmp_dict[cell['data-stat']] = float(val)
                    except:
                        tmp_dict[cell['data-stat']] = val
            tmp_dict['quarter'] = quarter
            tmp_dict['year'] = yr
            tmp_dict['week'] = wk
            tmp_dict['away_team'] = header[0]
            tmp_dict['home_team'] = header[1]
            pbp.append(tmp_dict)
    return pbp

In [141]:
pbp2 = pbp_extract(pg_url, 2017, 1)

In [ ]:
aaa = "Tom Brady pass incomplete deep middle intended for Rob Gronkowski. Penalty on Dee Ford: Defensive Offside, 5 yards (no play)"

In [142]:
pbp2

[{'qtr_time_remain': '15:00',
  'down': None,
  'yds_to_go': None,
  'location': 'KAN 35',
  'detail': 'Cairo Santos kicks off 64 yards, returned by Dion Lewis for 26 yards (tackle by Kevin Pierre-Louis)',
  'pbp_score_aw': 0.0,
  'pbp_score_hm': 0.0,
  'exp_pts_before': 0.0,
  'exp_pts_after': -0.74,
  'quarter': 1,
  'year': 2017,
  'week': 1,
  'away_team': 'Kansas City Chiefs',
  'home_team': 'New England Patriots'},
 {'qtr_time_remain': '14:55',
  'down': 1.0,
  'yds_to_go': 10.0,
  'location': 'NWE 27',
  'detail': 'Tom Brady pass incomplete deep left intended for Dwayne Allen',
  'pbp_score_aw': 0.0,
  'pbp_score_hm': 0.0,
  'exp_pts_before': 0.74,
  'exp_pts_after': 0.2,
  'quarter': 1,
  'year': 2017,
  'week': 1,
  'away_team': 'Kansas City Chiefs',
  'home_team': 'New England Patriots'},
 {'qtr_time_remain': '14:49',
  'down': 2.0,
  'yds_to_go': 10.0,
  'location': 'NWE 27',
  'detail': 'Tom Brady pass complete short right to Rex Burkhead for 8 yards (tackle by Ron Parker)'

In [119]:
pbp

[{'qtr_time_remain': '15:00',
  'down': None,
  'yds_to_go': None,
  'location': 'KAN 35',
  'detail': 'Cairo Santos kicks off 64 yards, returned by Dion Lewis for 26 yards (tackle by Kevin Pierre-Louis)',
  'pbp_score_aw': 0.0,
  'pbp_score_hm': 0.0,
  'exp_pts_before': 0.0,
  'exp_pts_after': -0.74,
  'quarter': 1},
 {'qtr_time_remain': '14:55',
  'down': 1.0,
  'yds_to_go': 10.0,
  'location': 'NWE 27',
  'detail': 'Tom Brady pass incomplete deep left intended for Dwayne Allen',
  'pbp_score_aw': 0.0,
  'pbp_score_hm': 0.0,
  'exp_pts_before': 0.74,
  'exp_pts_after': 0.2,
  'quarter': 1},
 {'qtr_time_remain': '14:49',
  'down': 2.0,
  'yds_to_go': 10.0,
  'location': 'NWE 27',
  'detail': 'Tom Brady pass complete short right to Rex Burkhead for 8 yards (tackle by Ron Parker)',
  'pbp_score_aw': 0.0,
  'pbp_score_hm': 0.0,
  'exp_pts_before': 0.2,
  'exp_pts_after': 0.56,
  'quarter': 1},
 {'qtr_time_remain': '14:14',
  'down': 3.0,
  'yds_to_go': 2.0,
  'location': 'NWE 35',
  'det

In [74]:
cells = rows[2].find_all('td')
len(cells)

9

In [73]:
int(cells[0].string[0])

1

In [77]:
cells[1].string

In [88]:
aa = cells[4].find_all('a')
aa[0]

<a name="pbp_1"></a>

In [93]:
aa

[<a name="pbp_1"></a>,
 <a href="/players/S/SantCa01.htm">Cairo Santos</a>,
 <a href="/players/L/LewiDi00.htm">Dion Lewis</a>,
 <a href="/players/P/PierKe00.htm">Kevin Pierre-Louis</a>]

In [104]:
cells[4].contents

[<a name="pbp_1"></a>,
 <a href="/players/S/SantCa01.htm">Cairo Santos</a>,
 ' kicks off 64 yards, returned by ',
 <a href="/players/L/LewiDi00.htm">Dion Lewis</a>,
 ' for 26 yards (tackle by ',
 <a href="/players/P/PierKe00.htm">Kevin Pierre-Louis</a>,
 ')']

In [110]:
''.join([x.string for x in cells[4].contents if x.string is not None])

'Cairo Santos kicks off 64 yards, returned by Dion Lewis for 26 yards (tackle by Kevin Pierre-Louis)'

In [103]:
[cells[4].contents[x].string for x in 

'Cairo Santos'

In [94]:
aa[3].string

'Kevin Pierre-Louis'

In [75]:
cells

[<td class="center " data-stat="qtr_time_remain"><a href="#pbp_1">15:00</a></td>,
 <td class="center " data-stat="down"></td>,
 <td class="center " data-stat="yds_to_go"></td>,
 <td class="left " csk="0" data-stat="location">KAN 35</td>,
 <td class="left " data-stat="detail"><a name="pbp_1"></a><a href="/players/S/SantCa01.htm">Cairo Santos</a> kicks off 64 yards, returned by <a href="/players/L/LewiDi00.htm">Dion Lewis</a> for 26 yards (tackle by <a href="/players/P/PierKe00.htm">Kevin Pierre-Louis</a>)</td>,
 <td class="right " data-stat="pbp_score_aw">0</td>,
 <td class="right " data-stat="pbp_score_hm">0</td>,
 <td class="right " data-stat="exp_pts_before">0.000</td>,
 <td class="right " data-stat="exp_pts_after">-0.740</td>]

# Put the code together

In [151]:
yr = 2017
wk = 1
pbp_list = []
gurls = game_urls(yr,wk)
for gurl in gurls:
    pbp = pbp_extract(gurl, yr, wk)
    pbp_list += pbp
    time.sleep(3)

In [152]:
df = pd.DataFrame.from_dict(pbp_list)
df.to_csv(f'{yr}_wk{wk}_v2.csv', index=False)

In [161]:
df.loc[11].detail

'Mike Gillislee middle for 2 yards, touchdown'

In [158]:
type(df.loc[8].detail)

str